# **🏠 데이터 결합**
> 아파트, 버스, 구별 방, 화장실 개수 합치기 

## Contents
- Library Import
- Data Load
- Data Merge


## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [15]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import geopy

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import re


## 2. Data Load

In [16]:
# 데이터 불러오기
train = pd.read_csv('../data/train_latlon_complete.csv')
test = pd.read_csv('../data/test_latlon_complete.csv')
apart = pd.read_csv('../data/전용면적최대_공간개수_대치_기준.csv')
subway = pd.read_csv('../data/지번주소별_유동인구_역세권_역까지거리_정류장까지거리.csv')

In [17]:
train.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')

In [19]:
# 결측치 함정카드 제거
train['중개사소재지'][train['중개사소재지'] == '-'] = np.nan
train['거래유형'][train['거래유형'] == '-'] = np.nan
train['등기신청일자'][train['등기신청일자'] == ' '] = np.nan

## 3. Data Merge

In [20]:
# 붙일 칼럼 만들기
train[['시', '구', '동']] = train['시군구'].str.split(' ', expand = True)
train['지번주소'] = train['시군구'] + ' ' + train['번지']
train['동아파트명'] = train['동'] + ' ' + train['아파트명']

test[['시', '구', '동']] = test['시군구'].str.split(' ', expand = True)
test['지번주소'] = test['시군구'] + ' ' + test['번지']
test['동아파트명'] = test['동'] + ' ' + test['아파트명']

In [21]:
# 겹치는 행 제외하고 불러오기
apart2 = apart[~apart.duplicated(['동아파트명', '전용면적2'])]

In [22]:
# 방, 화장실개수 붙여넣기 (공간개수 = 방 + 화장실)
train2 = pd.merge(train, apart2, how = 'left', left_on = ['동아파트명', '전용면적(㎡)'], right_on = ['동아파트명', '전용면적2'])
test2 = pd.merge(test, apart2, how = 'left', left_on = ['동아파트명', '전용면적(㎡)'], right_on = ['동아파트명', '전용면적2'])

In [23]:
# 좌표 버리기
subway2 = subway.drop(['좌표X뉴', '좌표Y뉴'], axis = 1)

In [24]:
# 역세권, 유동인구 붙여넣기 (유동 승객수 = 한달 단위 승하차평균, 역까지 거리 = 최단거리역 기준)
train3 = pd.merge(train2, subway2, how = 'left', on = '지번주소')
test3 = pd.merge(test2, subway2, how = 'left', on = '지번주소')

In [25]:
train3.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', '시', '구', '동', '지번주소', '동아파트명',
       '전용면적2', '방개수2', '화장실개수2', '공간개수', '유동승객수', '역까지_거리', '1차역세권', '2차역세권',
       'X좌표', 'Y좌표', '정류장까지_거리'],
      dtype='object')

In [26]:
# 필요없는 칼럼 버리기
# train3.drop(['본번', '부번', '등기신청일자', 'k-전화번호', 'k-팩스번호', '단지소개기존clob',
#             'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-사용검사일-사용승인일', 'k-연면적',
#             'k-주거전용면적', 'k-관리비부과면적', 'k-홈페이지', 'k-등록일자', 'k-수정일자',
#             '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
#             '단지승인일', '관리비 업로드', '사용허가여부'], axis = 1).isna().sum()

In [27]:
# 저장하기
train3.drop(['본번', '부번', '등기신청일자', 'k-전화번호', 'k-팩스번호', '단지소개기존clob',
             'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-사용검사일-사용승인일', 'k-연면적',
             'k-주거전용면적', 'k-관리비부과면적', 'k-홈페이지', 'k-등록일자', 'k-수정일자',
             '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
             '단지승인일', '관리비 업로드', '사용허가여부'], axis = 1).to_csv('../data/train_merge_sub_room.csv', index = False)
test3.drop(['본번', '부번', '등기신청일자', 'k-전화번호', 'k-팩스번호', '단지소개기존clob',
             'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-사용검사일-사용승인일', 'k-연면적',
             'k-주거전용면적', 'k-관리비부과면적', 'k-홈페이지', 'k-등록일자', 'k-수정일자',
             '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
             '단지승인일', '관리비 업로드', '사용허가여부'], axis = 1).to_csv('../data/test_merge_sub_room.csv', index = False)